In [1]:
import numpy as np
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [2]:
def get_data_loaders(train_batch_size, val_batch_size):
    data_transform = Compose([ToTensor(), Normalize((0.1307, ), (0.3081, ))])

    train_loader = DataLoader(MNIST(download=True,
                                    root="./MNIST_data",
                                    transform=data_transform,
                                    train=True),
                              batch_size=train_batch_size,
                              shuffle=True)

    val_loader = DataLoader(MNIST(download=False,
                                  root="./MNIST_data",
                                  transform=data_transform,
                                  train=False),
                            batch_size=val_batch_size,
                            shuffle=False)
    return train_loader, val_loader

In [3]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Conv2d(1, 16, 3, 1),
            nn.ReLU(True),
            nn.Conv2d(16, 32, 3, 1),
            nn.ReLU(True),
            nn.Conv2d(32, 64, 3, 1),
            nn.AdaptiveAvgPool2d(1),
        )
        self.layer.cuda()

    def forward(self, x):
        output = self.layer(x)
        output.squeeze_()
        return output

In [4]:
model = MyModel()
model.seq = torch.nn.DataParallel(model.seq)

In [35]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=1)

In [36]:
for epoch in range(10):
    avg_cost = 0

    for X, Y in data_loader:
        X = X.cuda()
        Y = Y.cuda()

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / 10

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 6.59890985
[Epoch:    2] cost = 1.49355698
[Epoch:    3] cost = 1.40849495
[Epoch:    4] cost = 1.27819133
[Epoch:    5] cost = 1.19299078
[Epoch:    6] cost = 1.20509815
[Epoch:    7] cost = 1.17073643
[Epoch:    8] cost = 1.16660511
[Epoch:    9] cost = 1.16047764
[Epoch:   10] cost = 1.15583706


In [37]:
model.eval()
correct = 0
for data, target in data_loader:
    data = data.cuda()
    target = target.cuda()
    # data, target = Variable(data, volatile=True), Variable(target)
    output = model(data)
    prediction = output.data.max(1)[1]
    correct += prediction.eq(target.data).sum()

print('Test set: Accuracy: {:.2f}%'.format(100. * correct / len(data_loader.dataset)))

Test set: Accuracy: 8.43%


In [34]:
model.eval()
correct = 0
for data, target in data_loader:
    data = data.cuda()
    target = target.cuda()
    # data, target = Variable(data, volatile=True), Variable(target)
    output = model(data)
    prediction = output.data.max(1)[1]
    correct += prediction.eq(target.data).sum()

print('Test set: Accuracy: {:.2f}%'.format(100. * correct / len(data_loader.dataset)))

Test set: Accuracy: 30.58%


In [21]:
y.data

tensor([7, 9, 1,  ..., 6, 8, 4], device='cuda:0')

In [22]:
y

tensor([7, 9, 1,  ..., 6, 8, 4], device='cuda:0')